In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [7]:
mnist.train.num_examples

55000

In [8]:
mnist.test.num_examples

10000

In [9]:
#initialize weights in filter
def initialize_weights (filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev=0.1)
    return(tf.Variable(init_random_dist))

In [10]:
#initialize bias
def initialize_bias (bias_shape):
    initial_bias_vals = tf.constant(0.1,shape=bias_shape)
    return(tf.Variable(initial_bias_vals))

In [11]:
#setup convolution layer and perform convolution computation: Dot Product(x*W)
def create_convolution_layer_and_compute_dot_product(inputs, filter_shape):
    #initialize the wieghts in the filter
    filter_initialized_with_weights=initialize_weights(filter_shape)
    
    #create a colvolution layer
    conv_layer_outputs=tf.nn.conv2d(inputs, filter_initialized_with_weights, strides=[1,1,1,1], padding=('SAME'))
    
    #return the convolution layer outputs
    return(conv_layer_outputs)

In [12]:
#setup a ReLU layer and perform computation: Dot Product + Bias (x.W + b)
def create_relu_layer_and_compute_dotproduct_plus_b(inputs, filter_shape):
    #initialize bias for each input channel
    b=initialize_bias([filter_shape[3]])
    
    #perform the computation first by adding: inputs (x*W) + b
    #create a ReLU layer associated with the preceeding convolution layer
    relu_layer_outputs=tf.nn.relu(inputs+b)
    
    #return the outputs of the relu layer
    return (relu_layer_outputs)

In [13]:
#setup a pooling layer and reduce the spacial size
def create_maxpool2by2_and_reduce_spacial_size(inputs):
    #create a pooling layer
    pooling_layer_outputs=tf.nn.max_pool(inputs,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    #return the pooling layer
    return (pooling_layer_outputs)

In [14]:
#setup fully connected layer and perform computation: (inputs*weights)+Bias
def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs,output_size):
    #inputs: the outputs from the preceding layer or previous operation like reshaping
    input_size=int(inputs.get_shape()[1])
    
    #each weight for one filter call
    W=initialize_weights([input_size, output_size])
    
    #initialize the bias: each bias one output channel
    b=initialize_bias([output_size])
    
    #First: Perform the computation for the FC layer: Dot product inputs*w
    fc_xW_plus_bias_outputs=tf.matmul(inputs,W)+b
    
    #return the results: outputs
    return(fc_xW_plus_bias_outputs)

In [15]:
#build convolutional neural network
x=tf.placeholder(tf.float32,shape=[None, 784])

In [16]:
#placeholder
y_true=tf.placeholder(tf.float32,[None,10])

In [17]:
#Reshape the Input Placeholder x
x_image=tf.reshape(x,[-1,28,28,1])

In [18]:
#create 1st convolutional layer
conv_layer_1_outputs=create_convolution_layer_and_compute_dot_product(x_image, filter_shape=[5,5,1,32])

conv_relu_layer_1_outputs=create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_1_outputs, filter_shape=[5,5,1,32])

In [19]:
pooling_layer_1_outputs=create_maxpool2by2_and_reduce_spacial_size(conv_relu_layer_1_outputs)

CREATE 2nd CONVOLUTION LAYER

In [20]:
#create 1st convolutional layer
conv_layer_2_outputs=create_convolution_layer_and_compute_dot_product(pooling_layer_1_outputs, filter_shape=[5,5,32,64])

#return the outputs of the layer
conv_relu_layer_2_outputs=create_relu_layer_and_compute_dotproduct_plus_b(conv_layer_2_outputs, filter_shape=[5,5,32,64])

CREATE 2nd POOLING LAYER

In [21]:
pooling_layer_2_outputs=create_maxpool2by2_and_reduce_spacial_size(conv_relu_layer_2_outputs)

RESHAPE/FLATTEN DATA MAKING IT READYTO BE FED INTO 1st FC LAYER

In [22]:
pooling_layer_2_outputs_flat=tf.reshape(pooling_layer_2_outputs, [-1, 7 * 7 *  64])

CREATE 1st FC LAYER, ReLU LAYER, and OUTPUT DATA TO DROPOUT LAYER

In [23]:
#return: outputs of the computation: (x * W) + b
fc_layer_1_outputs=create_fully_connected_layer_and_compute_dotproduct_plus_bias(pooling_layer_2_outputs_flat, output_size=1024)

#return: outputs of the layer
fc_relu_layer_1_outputs=tf.nn.relu(fc_layer_1_outputs)

CREATE DROPOUT LAYER AND DROPOUT A FRACTION OF OUTPUTS RANDOMLY 

In [24]:
#which modes/channels to keep or dropout: selected randomly
hold_prob=tf.placeholder(tf.float32)

#dropout
#set the outputs to 0 (zero) so that they will be ignored in the next layer
fc_dropout_outputs=tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


CREATE FINAL FC LAYER, COMPUTE (x.W+b), AND PRODUCE FINAL OUTPUTS

In [25]:
#return y_pred
y_pred=create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

DEFINE LOSS FUNCTION AND CALCULATE SOFTMAX CROSSENTROPY LOSS

In [26]:
#calculate the softmax cross-entropy loss
softmax_cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

#compute the mean of losses
cross_entropy_mean=tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



CREATE AN OPTIMIZER TO OPTIMIZE CNN MODEL AND SET LEARNING RATE

In [27]:
#get an ADAM optimizer
optimizer=tf.train.AdamOptimizer(learning_rate=0.001)

CREATE A TRAINER TO TRAIN CNN MODEL

In [28]:
cnn_trainer=optimizer.minimize(cross_entropy_mean)

TRAIN AND TEST CNN DEEP LEARNING MODEL ON MNIST DATASET

Create a Variable Initializer to Initialize All Variables

In [29]:
#get a variable initializer
vars_initializer=tf.global_variables_initializer()

Set the Steps

In [30]:
steps=5000

Run tf.Session() to train and test deep learning CNN model

In [31]:
with tf.Session() as sess:
    #First, run vars_initializer to initialize all variables
    sess.run(vars_initializer)
    for i in range(steps):
        #each batch: 50 images
        batch_x, batch_y=mnist.train.next_batch(50)
        
        #train the model
        #dropout keep_prob (% to keep):0.5 --> 50% will be dropped out
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})
        
        #test the model: at each 100th step
        #run the block of code for each 100 times of training, each time run a batch
        if i % 100 -- 0:
            print('ON STEP: {}'.format(i))
            print('ACCURACY: ')
            
            #COMPARE TO find matches of y_pred and y_true
            matches=tf.equal(tf.argmax(y_pred,axis=1),tf.argmax(y_true,axis=1))
            
            #cost the matches from integers to tf.float32
            #calculate the accuracy using the mean of matches
            acc=tf.reduce_mean(tf.cast(matches,tf.float32))
            
            #test the model at each 100th step
            #using test dataset
            #dropout: NONE because of the test , not training
            test_accuracy=sess.run(acc,feed_dict={x: mnist.test.images,y_true:mnist.test.labels,hold_prob: 1.0})
            
            print(test_accuracy)
            print('\n')
            

ON STEP: 1
ACCURACY: 
0.1785


ON STEP: 2
ACCURACY: 
0.2083


ON STEP: 3
ACCURACY: 
0.361


ON STEP: 4
ACCURACY: 
0.4174


ON STEP: 5
ACCURACY: 
0.5054


ON STEP: 6
ACCURACY: 
0.4641


ON STEP: 7
ACCURACY: 
0.4373


ON STEP: 8
ACCURACY: 
0.5405


ON STEP: 9
ACCURACY: 
0.5588


ON STEP: 10
ACCURACY: 
0.5248


ON STEP: 11
ACCURACY: 
0.5672


ON STEP: 12
ACCURACY: 
0.6258


ON STEP: 13
ACCURACY: 
0.6817


ON STEP: 14
ACCURACY: 
0.6934


ON STEP: 15
ACCURACY: 
0.7113


ON STEP: 16
ACCURACY: 
0.741


ON STEP: 17
ACCURACY: 
0.7667


ON STEP: 18
ACCURACY: 
0.7792


ON STEP: 19
ACCURACY: 
0.7751


ON STEP: 20
ACCURACY: 
0.7658


ON STEP: 21
ACCURACY: 
0.7829


ON STEP: 22
ACCURACY: 
0.8144


ON STEP: 23
ACCURACY: 
0.8275


ON STEP: 24
ACCURACY: 
0.8399


ON STEP: 25
ACCURACY: 
0.8569


ON STEP: 26
ACCURACY: 
0.8672


ON STEP: 27
ACCURACY: 
0.8705


ON STEP: 28
ACCURACY: 
0.8736


ON STEP: 29
ACCURACY: 
0.8771


ON STEP: 30
ACCURACY: 
0.8806


ON STEP: 31
ACCURACY: 
0.8781


ON STEP: 32
ACCURAC